In [139]:
import timeit
import time
import sys
# !{sys.executable} -m pip install lightgbm
from sklearn.preprocessing import MinMaxScaler, Imputer,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from contextlib import contextmanager
from sklearn.decomposition import PCA
#from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_predict, cross_val_score
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
from fancyimpute import KNN
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import missingno as mn

import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()


# from plotly import tools
# import plotly.tools as tls
# import squarify
# from mpl_toolkits.basemap import Basemap
# from numpy import array
# from matplotlib import cm

# import cufflinks and offline mode
import cufflinks as cf
cf.go_offline()


# # Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")

# # Print all rows and columns
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
% matplotlib inline
pd.set_option('display.max_rows', 500)
print(np.__version__)
pd.options.display.max_columns = 500


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


1.14.5


# 1. Reading Data into Pandas Dataframe:

![](~/.kaggle/competitions/home-credit-default-risk/IMG.jpg)

In [4]:
# Reading all csv files into a Pandas Dataframe
application_train= pd.read_csv("application_train.csv")
application_test=pd.read_csv("application_test.csv")
bureau_balance=pd.read_csv("bureau_balance.csv")
POS_CASH_balance=pd.read_csv("POS_CASH_balance.csv")
previous_application=pd.read_csv("previous_application.csv")
installments_payments=pd.read_csv("installments_payments.csv")
credit_card_balance=pd.read_csv("credit_card_balance.csv")
sample_submission=pd.read_csv("sample_submission.csv")
bureau=pd.read_csv("bureau.csv")
                             

In [3]:
#Exploring sizes of dataframes
print("Size of POS_CASH_BALANCE:", POS_CASH_balance.shape)
print("Size of bureau_balance:", bureau_balance.shape)
print("Size of application_train:", application_train.shape)
print("Size of previous_application:", previous_application.shape)
print("Size of installments_payments:", installments_payments.shape)
print("Size of credit_card_balance:", credit_card_balance.shape)
print("Size of sample_submission:", sample_submission.shape)
print("Size of application_test:", application_test.shape)
print("Size of bureau:", bureau.shape)

Size of POS_CASH_BALANCE: (10001358, 8)
Size of bureau_balance: (27299925, 3)
Size of application_train: (307511, 122)
Size of previous_application: (1670214, 37)
Size of installments_payments: (13605401, 8)
Size of credit_card_balance: (3840312, 23)
Size of sample_submission: (48744, 2)
Size of application_test: (48744, 121)
Size of bureau: (1716428, 17)


In [5]:
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
var

[('bureau_balance', 2238593954),
 ('previous_application', 2006318435),
 ('POS_CASH_balance', 1192493216),
 ('credit_card_balance', 918225044),
 ('installments_payments', 870745768),
 ('application_train', 570142137),
 ('bureau', 536987026),
 ('application_test', 90020943),
 ('sample_submission', 780008),
 ('Imputer', 1056),
 ('KFold', 1056),
 ('KNN', 1056),
 ('LogisticRegression', 1056),
 ('MinMaxScaler', 1056),
 ('PCA', 1056),
 ('StratifiedKFold', 1056),
 ('var', 488),
 ('Out', 240),
 ('color', 176),
 ('contextmanager', 136),
 ('cross_val_predict', 136),
 ('init_notebook_mode', 136),
 ('iplot', 136),
 ('roc_auc_score', 136),
 ('roc_curve', 136),
 ('In', 128),
 ('ipython_vars', 112),
 ('cf', 80),
 ('gc', 80),
 ('go', 80),
 ('matplotlib', 80),
 ('mn', 80),
 ('np', 80),
 ('offline', 80),
 ('pd', 80),
 ('plt', 80),
 ('py', 80),
 ('sns', 80),
 ('sys', 80),
 ('warnings', 80),
 ('get_ipython', 64),
 ('exit', 56),
 ('quit', 56)]

# 2. Glimpse of Data
Here, we check out the heads of each table.

In [4]:
application_train.head()

In [6]:
application_train.columns.values

In [7]:
application_test.head()

In [8]:
bureau.head()

In [9]:
bureau_balance.head()

In [10]:
previous_application.head()

In [11]:
POS_CASH_balance.head()

In [12]:
credit_card_balance.head()

In [13]:
installments_payments.head()

### 3. Data Preparation

#### Important Features:
  'REGION_POPULATION_RELATIVE',
 'FLAG_DOCUMENT_3',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'DAYS_LAST_PHONE_CHANGE',
 'ORGANIZATION_TYPE',
 'AMT_CREDIT',
 'AMT_GOODS_PRICE',
 'DAYS_REGISTRATION',
 'AMT_ANNUITY',
 'CODE_GENDER',
 'DAYS_ID_PUBLISH',
 'NAME_EDUCATION_TYPE',
 'DAYS_EMPLOYED',
 'DAYS_BIRTH',
 'EXT_SOURCE_1',
 'EXT_SOURCE_3',
 'EXT_SOURCE_2']
  0.0045129961860323579,
 0.0048575927947851092,
 0.0055161419341812111,
 0.0063633488627312584,
 0.0070533715778535074,
 0.0071525742205493271,
 0.0098334856799054057,
 0.010690091367919978,
 0.011025641227141648,
 0.013036317019276902,
 0.013514977295662424,
 0.014002708099649534,
 0.01844010404111723,
 0.020978691408358511,
 0.047694141280722489,
 0.070695366263430287,
 0.30960812640697144,
 0.33582739604990891]

#### Data Imputation
 - We remove the following features due to too many missing values:
    "COMMONAREA_AVG", "COMMONAREA_MODE", "NONLIVINGAPARTMENTS_MODE", "NONLIVINGAPARTMENTS_MEDI", "NONLIVINGAPARTMENTS_AVG", "FONDKAPREMONT_MODE", "LIVINGAPARTMENTS_MEDI", "LIVINGAPARTMENTS_AVG", "FLOORSMIN_MODE", "FLOORSMIN_AVG", "YEARS_BUILD_MEDI", "YEARS_BUILD_AVG", "YEARS_BUILD_MODE", "LANDAREA_MODE", "LANDAREA_AVG", "LANDAREA_MEDI", "BASEMENTAREA_MEDI", "BASEMENTAREA_AVG", "BASEMENTAREA_MODE", "NONLIVINGAREA_MEDI", "NONLIVINGAREA_AVG", "NONLIVINGAREA_MODE", "ELEVATORS_MODE", "ELEVATORS_AVG", "ELEVATORS_MEDI", "EMERGENCYSTATE_MODE", "TOTALAREA_MODE", "YEARS_BEGINEXPLUATATION_MODE", "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BEGINEXPLUATATION_MEDI", "FLOORSMAX_AVG", "FLOORSMAX_MEDI", "FLOORSMAX_MODE", "HOUSETYPE_MODE",  "LIVINGAREA_AVG", "LIVINGAREA_MODE", "LIVINGAREA_MEDI", "ENTRANCES_AVG", "ENTRANCES_MODE", ENTRANCES_MEDI", "APARTMENTS_AVG", "APARTMENTS_MODE", "APARTMENTS_MEDI", "WALLSMATERIAL_MODE"

In [5]:
columns = ["COMMONAREA_AVG", "COMMONAREA_MODE", "NONLIVINGAPARTMENTS_MODE", "NONLIVINGAPARTMENTS_MEDI", 
           "NONLIVINGAPARTMENTS_AVG", "FONDKAPREMONT_MODE", "LIVINGAPARTMENTS_MEDI", "LIVINGAPARTMENTS_AVG", 
           "FLOORSMIN_MODE", "FLOORSMIN_AVG", "YEARS_BUILD_MEDI", "YEARS_BUILD_AVG", "YEARS_BUILD_MODE", 
           "LANDAREA_MODE", "LANDAREA_AVG", "LANDAREA_MEDI", "BASEMENTAREA_MEDI", "BASEMENTAREA_AVG", 
           "BASEMENTAREA_MODE", "NONLIVINGAREA_MEDI", "NONLIVINGAREA_AVG", "NONLIVINGAREA_MODE", "ELEVATORS_MODE", 
           "ELEVATORS_AVG", "ELEVATORS_MEDI", "EMERGENCYSTATE_MODE", "TOTALAREA_MODE", "YEARS_BEGINEXPLUATATION_MODE", 
           "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BEGINEXPLUATATION_MEDI", "FLOORSMAX_AVG", "FLOORSMAX_MEDI", 
           "FLOORSMAX_MODE", "HOUSETYPE_MODE",  "LIVINGAREA_AVG", "LIVINGAREA_MODE", "LIVINGAREA_MEDI", "ENTRANCES_AVG",
           "ENTRANCES_MODE", "ENTRANCES_MEDI", "APARTMENTS_AVG", "APARTMENTS_MODE", "APARTMENTS_MEDI", 
           "WALLSMATERIAL_MODE","COMMONAREA_MEDI","LIVINGAPARTMENTS_MODE","FLOORSMIN_MEDI"]

 - EXT_SOURCE_1	173378	56.381073
 - EXT_SOURCE_3	60965	19.825307
 - AMT_REQ_CREDIT_BUREAU_YEAR	41519	13.501631
 - AMT_REQ_CREDIT_BUREAU_MON	41519	13.501631
 - AMT_REQ_CREDIT_BUREAU_WEEK	41519	13.501631
 - AMT_REQ_CREDIT_BUREAU_DAY	41519	13.501631
 - AMT_REQ_CREDIT_BUREAU_HOUR	41519	13.501631
 - AMT_REQ_CREDIT_BUREAU_QRT	41519	13.501631
 - NAME_TYPE_SUITE	1292	0.420148
 - OBS_30_CNT_SOCIAL_CIRCLE	1021	0.332021
 - DEF_30_CNT_SOCIAL_CIRCLE	1021	0.332021
 - OBS_60_CNT_SOCIAL_CIRCLE	1021	0.332021
 - DEF_60_CNT_SOCIAL_CIRCLE	1021	0.332021
 - EXT_SOURCE_2	660	0.214626
 - AMT_GOODS_PRICE	278	0.090403
 - AMT_ANNUITY	12	0.003902
 - CNT_FAM_MEMBERS	2	0.000650
 - DAYS_LAST_PHONE_CHANGE	1	0.000325

 - Creating features from bureau:
 - Credit_active: n_closed, n_bad, n_sold , n_active, n_closed/(n_sold+n_bad), n_active/n_closed, 
 - Credit_currency: n_1, n_2, n_3
 - days_credit: min, max, mean, var
 - CREDIT_DAY_OVERDUE: 
 - 'AMT_CREDIT_MAX_OVERDUE': ['mean'],
 - 'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
 - 'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
 - 'AMT_CREDIT_SUM_OVERDUE': ['mean'],
 - 'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
 - 'AMT_ANNUITY': ['max', 'mean'],
 - 'CNT_CREDIT_PROLONG': ['sum'],
 -'MONTHS_BALANCE_MIN': ['min'],
 - 'MONTHS_BALANCE_MAX': ['max'],
 - 'MONTHS_BALANCE_SIZE': ['mean', 'sum']

In [5]:
n_train=application_train.shape[0]
n_train

307511

In [196]:
def var_memory():
    # These are the usual ipython objects, including this one you are creating
    ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
    # Get a sorted list of the objects and their sizes
    var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
    return var
def missing_value_summary(df): 
    total= df.isnull().sum().sort_values(ascending = False)
    percentage= (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    missing_df_data=pd.concat([total,percentage], axis=1, keys=['Total','Percent'])
    del df
    gc.collect()
    return missing_df_data

def missing_value_vis(df):
    mv_vis=mn.matrix(df)
    del df
    gc.collect()
    return mv_vis

def imputation(df, columns):
    df= df.drop(columns, axis=1)
    occupation_mode= df["OCCUPATION_TYPE"].value_counts()[0]
    #We impute with most common occupation for now
    df["OCCUPATION_TYPE"]=df["OCCUPATION_TYPE"].fillna(value=occupation_mode)
    
    #For people who do not own cars, we impute OWN_CAR_AGE with 0
    m= (df["OWN_CAR_AGE"].isnull())&(df["FLAG_OWN_CAR"]=="N") 
    df.loc[m,"OWN_CAR_AGE"]=df.loc[m,"OWN_CAR_AGE"].fillna(0)
    own_car_median= df["OWN_CAR_AGE"].median()
    df["OWN_CAR_AGE"].fillna(own_car_median, inplace=True)
    
    # imputing with median
    df[["EXT_SOURCE_1", "EXT_SOURCE_3", "AMT_REQ_CREDIT_BUREAU_YEAR", "AMT_REQ_CREDIT_BUREAU_MON", 
        "AMT_REQ_CREDIT_BUREAU_WEEK", "AMT_REQ_CREDIT_BUREAU_DAY", "AMT_REQ_CREDIT_BUREAU_HOUR", 
        "AMT_REQ_CREDIT_BUREAU_QRT", "OBS_30_CNT_SOCIAL_CIRCLE", "DEF_30_CNT_SOCIAL_CIRCLE",
       "OBS_60_CNT_SOCIAL_CIRCLE","DEF_60_CNT_SOCIAL_CIRCLE", "EXT_SOURCE_2", "AMT_GOODS_PRICE",
       "AMT_ANNUITY","CNT_FAM_MEMBERS","DAYS_LAST_PHONE_CHANGE"]] = df[["EXT_SOURCE_1", "EXT_SOURCE_3", "AMT_REQ_CREDIT_BUREAU_YEAR", "AMT_REQ_CREDIT_BUREAU_MON", 
        "AMT_REQ_CREDIT_BUREAU_WEEK", "AMT_REQ_CREDIT_BUREAU_DAY", "AMT_REQ_CREDIT_BUREAU_HOUR", 
        "AMT_REQ_CREDIT_BUREAU_QRT", "OBS_30_CNT_SOCIAL_CIRCLE", "DEF_30_CNT_SOCIAL_CIRCLE",
       "OBS_60_CNT_SOCIAL_CIRCLE","DEF_60_CNT_SOCIAL_CIRCLE", "EXT_SOURCE_2", "AMT_GOODS_PRICE",
       "AMT_ANNUITY","CNT_FAM_MEMBERS","DAYS_LAST_PHONE_CHANGE"]].apply(lambda x : x.fillna(x.median()), axis=0)
    name_type_mode=df["NAME_TYPE_SUITE"].value_counts()[0]
    df["NAME_TYPE_SUITE"].fillna(name_type_mode, inplace=True)
    
    gc.collect()
    
    #imputing EXT_SOURCE_1 and EXT_SOURCE_3
    return df


def delete_columns(df,columns):
    return df.drop(columns, axis=1)

def impute_median(df, columns):
    medians= df[columns].median()
    for col in columns:
        df[[col]].fillna(medians[col], inplace=True)
    #df[columns]= df[columns].apply( lambda x: x.fillna(x.median()), axis=1 )
    gc.collect()
    return df

def feature_engineering_bureau_balance(df):
    cat_aggregations={
        'STATUS_0':["sum"],
        'STATUS_1':["sum"],
        'STATUS_2':["sum"],
        'STATUS_3':["sum"],
        'STATUS_4':["sum"],
        'STATUS_5':["sum"],
        'STATUS_C':["sum"],
        'STATUS_X':["sum"]
    }
    aggdf= df.groupby("SK_ID_BUREAU").agg(cat_aggregations)
    aggdf.columns=["BB"+"_"+e[0]+"_"+e[1] for e in aggdf.columns]
    del df
    gc.collect()
    return aggdf
def feature_engineering_bureau(df):
    num_aggregations={
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'CREDIT_DAY_OVERDUE':['sum', 'mean','max'],
        'DAYS_CREDIT_ENDDATE': ['mean'],
        'DAYS_ENDDATE_FACT':['mean','max', 'min'],
        'CNT_CREDIT_PROLONG': ['sum', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean','max'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        #'MONTHS_BALANCE_MIN': ['min'],
        #'MONTHS_BALANCE_MAX': ['max'],
        #'MONTHS_BALANCE_SIZE': ['mean', 'sum']
        
    }
    cat_aggregations={
        #We can also create good to bad loan ratios
        'CREDIT_ACTIVE_Bad debt':["sum","mean"],
        'CREDIT_ACTIVE_Closed':["sum","mean"],
        'CREDIT_ACTIVE_Sold':["sum","mean"],
        'CREDIT_CURRENCY_currency 2':["sum","mean"],
        'CREDIT_CURRENCY_currency 3':["sum","mean"],
        'CREDIT_CURRENCY_currency 4':["sum","mean"],
        'CREDIT_TYPE_Car loan':["sum","mean"],
        'CREDIT_TYPE_Cash loan (non-earmarked)':["sum","mean"],
        'CREDIT_TYPE_Consumer credit':["sum","mean"],
        'CREDIT_TYPE_Credit card':["sum","mean"],
        'CREDIT_TYPE_Interbank credit':["sum","mean"],
        'CREDIT_TYPE_Loan for business development':["sum","mean"],
        'CREDIT_TYPE_Loan for purchase of shares (margin lending)':["sum","mean"],
        'CREDIT_TYPE_Loan for the purchase of equipment':["sum","mean"],
        'CREDIT_TYPE_Loan for working capital replenishment':["sum","mean"],
        'CREDIT_TYPE_Microloan':["sum","mean"],
        'CREDIT_TYPE_Mobile operator loan':["sum","mean"],
        'CREDIT_TYPE_Mortgage':["sum","mean"],
        'CREDIT_TYPE_Real estate loan':["sum","mean"],
        'CREDIT_TYPE_Unknown type of loan':["sum","mean"],
        'BB_STATUS_0_sum':["sum","mean"],
        'BB_STATUS_1_sum':["sum","mean"],
        'BB_STATUS_2_sum':["sum","mean"],
        'BB_STATUS_3_sum':["sum","mean"],
        'BB_STATUS_4_sum':["sum","mean"],
        'BB_STATUS_5_sum':["sum","mean"],
        'BB_STATUS_C_sum':["sum","mean"],
        'BB_STATUS_X_sum':["sum","mean"]
    }
    aggdf=df.groupby("SK_ID_CURR").agg({**num_aggregations, **cat_aggregations})
    aggdf.columns = ["BUREAU_"+e[0]+"_"+e[1] for e in aggdf.columns]
    del df
    gc.collect()
    return aggdf

def one_hot_encoder(df, nan_as_category=True, drop_first=True):
    original_columns = list(df.columns)
    categorical_columns=[col for col in original_columns if df[col].dtype=='object']
    df=pd.get_dummies(data=df, columns=categorical_columns, dummy_na=nan_as_category, drop_first=drop_first)
    new_columns = [c for c in df.columns if c not in original_columns]
    gc.collect()
    return df, new_columns

def feature_engineering_previous(df, cat_cols):
    
    num_aggregations = {
        'AMT_ANNUITY': ['min','max','mean'],
        'AMT_APPLICATION': ['min','max','mean'],
        'AMT_CREDIT': ['min','max','mean'],
        'APP_CREDIT_PERC': ['min','max','mean'],
        'AMT_DOWN_PAYMENT': ['min','max','mean'],        
        'AMT_GOODS_PRICE': ['min','max','mean'],
        'HOUR_APPR_PROCESS_START': ['min','max','mean'],
        'RATE_DOWN_PAYMENT': ['min','max','mean'],  
        'DAYS_DECISION': ['min','max','mean'],
        'CNT_PAYMENT': ['min','sum']
    }
    cat_aggregations=dict()
    for cat in cat_cols:
        cat_aggregations[cat]= ['mean', 'sum']
    aggdf=df.groupby("SK_ID_CURR").agg({**num_aggregations, **cat_aggregations})
    aggdf.columns = ["PREV_"+e[0]+"_"+e[1] for e in aggdf.columns]
    del df
    gc.collect()
    return aggdf

def feature_engineering_POS_CASH_balance(df, cat_cols):
    num_aggregations = {
        'MONTHS_BALANCE':['max', 'min', 'size'],
        'SK_DPD':['max', 'min','mean'],
        'SK_DPD_DEF':['max', 'min','mean']
    }
    cat_aggregations={}
    for col in cat_cols:
        cat_aggregations[col]= ['sum','mean']
    aggdf= df.groupby("SK_ID_CURR").agg({**num_aggregations, **cat_aggregations})
    aggdf.columns = ["POS_"+e[0]+"_"+e[1] for e in aggdf.columns]
    del df
    gc.collect()
    return aggdf

def feature_engineering_installments(df):
    df['DAY_DIFF'] = df['DAYS_ENTRY_PAYMENT'] = df['DAYS_INSTALMENT']
    df['TIMELY'] = df['DAY_DIFF'].apply(lambda x: 1 if x>0 else 0)
    df['PAY_PERC'] = df['AMT_PAYMENT']/df['AMT_INSTALMENT']
    df['PAY_DIFF'] = df['AMT_INSTALMENT'] - df['AMT_PAYMENT']
    aggregations= {
        'TIMELY':['sum','mean'],
        'PAY_PERC':['max', 'mean', 'sum','var'],
        'PAY_DIFF':['max', 'mean', 'sum','var'],
        'AMT_PAYMENT':['max','mean','sum'],
        'AMT_INSTALMENT':['max', 'mean', 'sum'],
        'DAY_DIFF':['min','max', 'mean','sum']
        
    }
    aggdf=df.groupby('SK_ID_CURR').agg(aggregations)
    aggdf.columns= ["INS_"+e[0]+"_"+e[1] for e in aggdf.columns]
    del df
    gc.collect()
    return aggdf


def feature_engineering_credit(df,cv=5):
    aggdf=df.groupby('SK_ID_CURR').agg(['min','max', 'sum','var','mean'])
    aggdf.columns=["CC_"+e[0]+"_"+e[1] for e in aggdf.columns]
    aggdf["CC_COUNT"] = df.groupby("SK_ID_CURR").size()
    del df
    gc.collect()
    return aggdf 

def preprocessing_imputer(df):
    #train= df.iloc[:n_train, :]
    train_X = df.iloc[:n_train, :].drop(['TARGET','SK_ID_CURR', 'index'],axis=1)
    train_y = df.iloc[:n_train, :].TARGET
    #test= df.iloc[n_train:,:]
    test_X=df.iloc[n_train:,:].drop(['TARGET','SK_ID_CURR', 'index'],axis=1)
    imputer= Imputer(strategy='median')
    imputer.fit(train_X)
    train_X=imputer.transform(train_X)
    test_X=imputer.transform(test_X)
    del df
    gc.collect()
    return train_X, test_X, train_y
    
def preprocessing_scaler(train_X,test_X):    
    scaler=StandardScaler()
    scaler.fit(train_X)
    train_X=scaler.transform(train_X)
    test_X=scaler.transform(test_X)
    gc.collect()
    return train_X, test_X

def logistic_regression(train_X, train_y, cv=5):
    lr= LogisticRegression(class_weight='balanced')
    score=cross_val_score(lr, train_X,train_y,cv=cv, scoring='roc_auc')
    gc.collect()
    return score

def random_forest(train_X, train_y,cv=5):
    rf= RandomForestClassifier(n_estimators=100, random_state=50)
    rf.fit(train_X, train_y)
    return rf

def feature_importance(model, features):
    plt.figure(figsize=(12,8))
    res = pd.DataFrame({'feature':features, 'importance': model.feature_importances_})
    res=res.sort_values('importance', ascending= False)
    print(res.head(10))
    print("Features > 0.01: ", np.sum(res['importance']> 0.01))
    
    res.head(20).plot(x='feature', y='importance', kind='barh',
                     color='red', edgecolor='k', title='Feature Importances')
    return res

In [15]:
missing_application_train=missing_value_summary(application_train)
missing_application_train


In [18]:
missing_value_vis(application_train.iloc[:,0:50])

In [19]:
missing_value_vis(application_train.iloc[:,50:100])

In [20]:
missing_value_vis(application_train.iloc[:,100:])

#### Data Imputation
 - We remove the following features due to too many missing values:
    "COMMONAREA_AVG", "COMMONAREA_MODE", "NONLIVINGAPARTMENTS_MODE", "NONLIVINGAPARTMENTS_MEDI", "NONLIVINGAPARTMENTS_AVG", "FONDKAPREMONT_MODE", "LIVINGAPARTMENTS_MEDI", "LIVINGAPARTMENTS_AVG", "FLOORSMIN_MODE", "FLOORSMIN_AVG", "YEARS_BUILD_MEDI", "YEARS_BUILD_AVG", "YEARS_BUILD_MODE", "LANDAREA_MODE", "LANDAREA_AVG", "LANDAREA_MEDI", "BASEMENTAREA_MEDI", "BASEMENTAREA_AVG", "BASEMENTAREA_MODE", "NONLIVINGAREA_MEDI", "NONLIVINGAREA_AVG", "NONLIVINGAREA_MODE", "ELEVATORS_MODE", "ELEVATORS_AVG", "ELEVATORS_MEDI", "EMERGENCYSTATE_MODE", "TOTALAREA_MODE", "YEARS_BEGINEXPLUATATION_MODE", "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BEGINEXPLUATATION_MEDI", "FLOORSMAX_AVG", "FLOORSMAX_MEDI", "FLOORSMAX_MODE", "HOUSETYPE_MODE",  "LIVINGAREA_AVG", "LIVINGAREA_MODE", "LIVINGAREA_MEDI", "ENTRANCES_AVG", "ENTRANCES_MODE", ENTRANCES_MEDI", "APARTMENTS_AVG", "APARTMENTS_MODE", "APARTMENTS_MEDI", "WALLSMATERIAL_MODE"

In [7]:
columns=['APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAREA_MEDI',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'TOTALAREA_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [8]:
application_train_imp = imputation(application_train, columns)

In [16]:
application_train_imp.columns

In [11]:
missing_value_vis(application_train_imp.iloc[:,40:])

In [25]:
missing_value_summary(application_train_imp)

Now, we perform one-hot encoding for categorical variables:

In [9]:
application_train_one_hot, application_cat = one_hot_encoder(application_train_imp, False )

In [17]:
application_train_one_hot.head()

In [18]:
application_train_one_hot.columns

In [10]:
del application_train_imp
gc.collect()

697

In [73]:
missing_value_summary(application_test)

In [12]:
application_test_imp = imputation(application_test,columns)

In [75]:
missing_value_summary(application_test_imp)

In [76]:
len(application_test_imp.columns)

In [13]:
application_test_one_hot, application_test_cat=one_hot_encoder(application_test_imp, False)

In [78]:
application_test_one_hot.head()

In [79]:
application_test_one_hot.shape[1]

In [14]:
atrcols=list(application_train_one_hot)
atcols=list(application_test_one_hot)
notcols= [c for c in atrcols if c not in atcols]
notcols
for col in notcols:
    if col!="TARGET":
        application_test_one_hot[col]=0
    else:
        application_test_one_hot[col]=np.nan
        
    

In [15]:
combined_df= application_train_one_hot.append(application_test_one_hot)\
.reset_index()

In [16]:
del application_test_imp,application_test_one_hot, application_train_one_hot,
application_train, application_test
gc.collect()

723

In [17]:
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
var

[('bureau_balance', 3006847673),
 ('previous_application', 2006318435),
 ('POS_CASH_balance', 1192493216),
 ('credit_card_balance', 918225044),
 ('installments_payments', 870745768),
 ('application_train', 570142137),
 ('bureau', 536987026),
 ('combined_df', 230497089),
 ('application_test', 90020943),
 ('sample_submission', 780008),
 ('atrcols', 1704),
 ('atcols', 1664),
 ('application_cat', 1072),
 ('application_test_cat', 1072),
 ('Imputer', 1056),
 ('KFold', 1056),
 ('KNN', 1056),
 ('LogisticRegression', 1056),
 ('MinMaxScaler', 1056),
 ('PCA', 1056),
 ('StandardScaler', 1056),
 ('StratifiedKFold', 1056),
 ('columns', 440),
 ('In', 264),
 ('Out', 240),
 ('color', 176),
 ('contextmanager', 136),
 ('cross_val_predict', 136),
 ('cross_val_score', 136),
 ('delete_columns', 136),
 ('feature_engineering_POS_CASH_balance', 136),
 ('feature_engineering_bureau', 136),
 ('feature_engineering_bureau_balance', 136),
 ('feature_engineering_credit', 136),
 ('feature_engineering_installments', 13

In [18]:
combined_df.head()

,index,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_M,CODE_GENDER_XNA,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,FLAG_PHONE,FLAG_WORK_PHONE,HOUR_APPR_PROCESS_START,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGA

Processing *bureau_balance* and *bureau* and joining them to the main dataset.

In [27]:
missing_value_summary(bureau_balance)

In [28]:
bureau_balance.STATUS.value_counts()

In [19]:
bureau_balance_dummy, bureau_balance_dummy_cat=one_hot_encoder(bureau_balance,True,False)

In [30]:
bureau_balance_dummy.head()

In [20]:
bureau_balance_dummy_agg= feature_engineering_bureau_balance(bureau_balance_dummy)

In [32]:
bureau_balance_dummy_agg.head()

In [33]:
missing_value_summary(bureau)

In [34]:
bureau.head()

In [35]:
bureau["CREDIT_ACTIVE"].value_counts()

In [36]:
missing_value_vis(bureau)

In [37]:
missing_value_summary(bureau)

In [22]:
bureau_del= delete_columns(bureau, ["AMT_ANNUITY", "AMT_CREDIT_MAX_OVERDUE"])

In [39]:
missing_value_summary(bureau_del)

#### Imputation Strategy:
 - DAYS_ENDDATE_FACT: Median
 - AMT_CREDIT_SUM_LIMIT: Median
 - DAYS_CREDIT_ENDDATE: Median
 

In [23]:
bureau_imp=impute_median(bureau_del,["DAYS_ENDDATE_FACT","AMT_CREDIT_SUM_LIMIT",
                                     "DAYS_CREDIT_ENDDATE","AMT_CREDIT_SUM_DEBT","AMT_CREDIT_SUM"])

In [41]:
missing_value_summary(bureau_imp)

In [24]:
bureau_imp_dummy, bureau_imp_dummy_cat= one_hot_encoder(bureau_imp, False,False)

In [25]:
bureau__leftjoin_bureaubalance = bureau_imp_dummy.join(bureau_balance_dummy_agg, how='left', on='SK_ID_BUREAU')

In [26]:
bureau__leftjoin_bureaubalance.head()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,BB_STATUS_0_sum,BB_STATUS_1_sum,BB_STATUS_2_sum,BB_STATUS_3_sum,BB_STATUS_4_sum,BB_STATUS_5_sum,BB_STATUS_C_sum,BB_STATUS_X_sum
0,215354,5714462,-497,0,-153.0,-153.0,0,91323.0,0.0,NaN,0.0,-131,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,-208,0,1075.0,NaN,0,225000.0,171342.0,NaN,0.0,-20,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,-203,0,528.0,NaN,0,464323.5,NaN,NaN,0.0,-16,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,-203,0,NaN,NaN,0,90000.0,NaN,NaN,0.0,-16,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,-629,0,1197.0,NaN,0,2700000.0,NaN,NaN,0.0,-21,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
bureau_agg= feature_engineering_bureau(bureau__leftjoin_bureaubalance)

In [28]:
bureau_agg.head()

,BUREAU_DAYS_CREDIT_min,BUREAU_DAYS_CREDIT_max,BUREAU_DAYS_CREDIT_mean,BUREAU_DAYS_CREDIT_var,BUREAU_CREDIT_DAY_OVERDUE_max,BUREAU_CREDIT_DAY_OVERDUE_mean,BUREAU_DAYS_CREDIT_ENDDATE_mean,BUREAU_DAYS_ENDDATE_FACT_mean,BUREAU_DAYS_ENDDATE_FACT_max,BUREAU_DAYS_ENDDATE_FACT_min,BUREAU_CNT_CREDIT_PROLONG_sum,BUREAU_CNT_CREDIT_PROLONG_mean,BUREAU_DAYS_CREDIT_UPDATE_mean,BUREAU_AMT_CREDIT_SUM_max,BUREAU_AMT_CREDIT_SUM_mean,BUREAU_AMT_CREDIT_SUM_sum,BUREAU_AMT_CREDIT_SUM_DEBT_max,BUREAU_AMT_CREDIT_SUM_DEBT_mean,BUREAU_AMT_CREDIT_SUM_DEBT_sum,BUREAU_AMT_CREDIT_SUM_OVERDUE_mean,BUREAU_AMT_CREDIT_SUM_OVERDUE_max,BUREAU_AMT_CREDIT_SUM_LIMIT_mean,BUREAU_AMT_CREDIT_SUM_LIMIT_sum,BUREAU_CREDIT_ACTIVE_Bad debt_sum,BUREAU_CREDIT_ACTIVE_Bad debt_mean,BUREAU_CREDIT_ACTIVE_Closed_sum,BUREAU_CREDIT_ACTIVE_Closed_mean,BUREAU_CREDIT_ACTIVE_Sold_sum,BUREAU_CREDIT_ACTIVE_Sold_mean,BUREAU_CREDIT_CURRENCY_currency 2_sum,BUREAU_CREDIT_CURRENCY_currency 2_mean,BUREAU_CREDIT_CURRENCY_currency 3_sum,BUREAU_CREDIT_CURRENCY_currency 3_mean,BUREAU_CREDIT_CURRENCY_currency 4_sum,BUREAU_CREDIT_CURRENCY_currency 4_mean,BUREAU_CREDIT_TYPE_Car loan_sum,BUREAU_CREDIT_TYPE_Car loan_mean,BUREAU_CREDIT_TYPE_Cash loan (non-earmarked)_sum,BUREAU_CREDIT_TYPE_Cash loan (non-earmarked)_mean,BUREAU_CREDIT_TYPE_Consumer credit_sum,BUREAU_CREDIT_TYPE_Consumer credit_mean,BUREAU_CREDIT_TYPE_Credit card_sum,BUREAU_CREDIT_TYPE_Credit card_mean,BUREAU_CREDIT_TYPE_Interbank credit_sum,BUREAU_CREDIT_TYPE_Interbank credit_mean,BUREAU_CREDIT_TYPE_Loan for business development_sum,BUREAU_CREDIT_TYPE_Loan for business development_mean,BUREAU_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,BUREAU_CREDIT_TYPE_Loan for purchase of shares (margin lending)_mean,BUREAU_CREDIT_TYPE_Loan for the purchase of equipment_sum,BUREAU_CREDIT_TYPE_Loan for the purchase of equipment_mean,BUREAU_CREDIT_TYPE_Loan for working capital replenishment_sum,BUREAU_CREDIT_TYPE_Loan for working capital replenishment_mean,BUREAU_CREDIT_TYPE_Microloan_sum,BUREAU_CREDIT_TYPE_Microloan_mean,BUREAU_CREDIT_TYPE_Mobile operator loan_sum,BUREAU_CREDIT_TYPE_Mobile operator loan_mean,BUREAU_CREDIT_TYPE_Mortgage_sum,BUREAU_CREDIT_TYPE_Mortgage_mean,BUREAU_CREDIT_TYPE_Real estate loan_sum,BUREAU_CREDIT_TYPE_Real estate loan_mean,BUREAU_CREDIT_TYPE_Unknown type of loan_sum,BUREAU_CREDIT_TYPE_Unknown type of loan_mean,BUREAU_BB_STATUS_0_sum_sum,BUREAU_BB_STATUS_0_sum_mean,BUREAU_BB_STATUS_1_sum_sum,BUREAU_BB_STATUS_1_sum_mean,BUREAU_BB_STATUS_2_sum_sum,BUREAU_BB_STATUS_2_sum_mean,BUREAU_BB_STATUS_3_sum_sum,BUREAU_BB_STATUS_3_sum_mean,BUREAU_BB_STATUS_4_sum_sum,BUREAU_BB_STATUS_4_sum_mean,BUREAU_BB_STATUS_5_sum_sum,BUREAU_BB_STATUS_5_sum_mean,BUREAU_BB_STATUS_C_sum_sum,BUREAU_BB_STATUS_C_sum_mean,BUREAU_BB_STATUS_X_sum_sum,BUREAU_BB_STATUS_X_sum_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,240043.666667,0,0.0,82.428571,-825.500000,-544.0,-1328.0,0,0.0,-93.142857,378000.0,207623.571429,1453365.000,373239.0,85240.928571,596686.5,0.0,0.0,0.00000,0.000,0,0.0,4,0.571429,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,7,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,31.0,4.428571,1.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,15.714286,30.0,4.285714
100002,-1437,-103,-874.000000,186150.000000,0,0.0,-349.000000,-697.500000,-36.0,-1185.0,0,0.0,-499.875000,450000.0,108131.945625,865055.565,245781.0,49156.200000,245781.0,0.0,0.0,7997.14125,31988.565,0,0.0,6,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,4,0.500000,4,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,45.0,5.625000,27.0,3.375000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,2.875000,15.0,1.875000
100003,-2586,-606,-1400.750000,827783.583333,0,0.0,-544.500000,-1097.333333,-540.0,-2131.0,0,0.0,-816.000000,810000.0,254350.125000,1017400.500,0.0,0.000000,0.0,0.0,0.0,202500.00000,810000.000,0,0.0,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2,0.500000,2,0

In [67]:
missing_value_summary(bureau_agg)

In [27]:
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
var

[('bureau_balance', 2238593954),
 ('previous_application', 2006318435),
 ('POS_CASH_balance', 1192493216),
 ('credit_card_balance', 918225044),
 ('installments_payments', 870745768),
 ('bureau_balance_dummy', 682498229),
 ('application_train', 570142137),
 ('bureau', 536987026),
 ('bureau_del', 509524178),
 ('bureau_imp', 509524178),
 ('bureau__leftjoin_bureaubalance', 314106428),
 ('combined_df', 230497089),
 ('bureau_imp_dummy', 204255036),
 ('bureau_agg', 152905524),
 ('application_test', 90020943),
 ('bureau_balance_dummy_agg', 13078344),
 ('sample_submission', 780008),
 ('atrcols', 1704),
 ('atcols', 1664),
 ('application_cat', 1072),
 ('application_test_cat', 1072),
 ('Imputer', 1056),
 ('KFold', 1056),
 ('KNN', 1056),
 ('LogisticRegression', 1056),
 ('MinMaxScaler', 1056),
 ('PCA', 1056),
 ('StratifiedKFold', 1056),
 ('var', 720),
 ('columns', 440),
 ('In', 344),
 ('bureau_imp_dummy_cat', 264),
 ('Out', 240),
 ('bureau_balance_dummy_cat', 192),
 ('color', 176),
 ('contextmanager

In [29]:
combined_bb=combined_df.join(bureau_agg, how="left", on="SK_ID_CURR")      

In [30]:
combined_bb["DAYS_EMPLOYED"].replace(365243, np.nan, inplace=True)

In [72]:
combined_bb.head()

In [31]:
del bureau__leftjoin_bureaubalance,bureau_agg,


In [32]:
del bureau_balance_dummy ,bureau_balance_dummy_agg, bureau_del, bureau_imp,bureau_imp_dummy, combined_df, bureau, bureau_balance
gc.collect()

2212

In [33]:
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
var

[('previous_application', 2006318435),
 ('POS_CASH_balance', 1192493216),
 ('credit_card_balance', 918225044),
 ('installments_payments', 870745768),
 ('application_train', 570142137),
 ('combined_bb', 455650249),
 ('application_test', 90020943),
 ('sample_submission', 780008),
 ('bureau_balance_dummy_cat', 1928),
 ('atrcols', 1704),
 ('atcols', 1664),
 ('application_cat', 1072),
 ('application_test_cat', 1072),
 ('Imputer', 1056),
 ('KFold', 1056),
 ('KNN', 1056),
 ('LogisticRegression', 1056),
 ('MinMaxScaler', 1056),
 ('PCA', 1056),
 ('StandardScaler', 1056),
 ('StratifiedKFold', 1056),
 ('var', 728),
 ('columns', 440),
 ('Out', 368),
 ('In', 344),
 ('bureau_imp_dummy_cat', 264),
 ('color', 176),
 ('contextmanager', 136),
 ('cross_val_predict', 136),
 ('cross_val_score', 136),
 ('delete_columns', 136),
 ('feature_engineering_POS_CASH_balance', 136),
 ('feature_engineering_bureau', 136),
 ('feature_engineering_bureau_balance', 136),
 ('feature_engineering_credit', 136),
 ('feature_en

#### Feature creation
CREDIT_ACTIVE: closed, active, bad, sold numbers



 - Creating features from bureau:
 - Credit_active: n_closed, n_bad, n_sold , n_active, n_closed/(n_sold+n_bad), n_active/n_closed, 
 - Credit_currency: n_1, n_2, n_3
 - days_credit: min, max, mean, var
 - CREDIT_DAY_OVERDUE: 
 - 'AMT_CREDIT_MAX_OVERDUE': ['mean'],
 - 'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
 - 'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
 - 'AMT_CREDIT_SUM_OVERDUE': ['mean'],
 - 'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
 - 'AMT_ANNUITY': ['max', 'mean'],
 - 'CNT_CREDIT_PROLONG': ['sum'],
 -'MONTHS_BALANCE_MIN': ['min'],
 - 'MONTHS_BALANCE_MAX': ['max'],
 - 'MONTHS_BALANCE_SIZE': ['mean', 'sum']

In [34]:
previous_application["APP_CREDIT_PERC"] =\
previous_application["AMT_APPLICATION"]/previous_application["AMT_CREDIT"]

In [80]:
missing_value_summary(previous_application)

In [81]:
previous_application.describe()

In [82]:
missing_value_vis(previous_application)

In [35]:
previous_application_one_hot, previous_application_cat = one_hot_encoder(previous_application)

In [36]:
previous_agg= feature_engineering_previous(previous_application_one_hot,previous_application_cat)

In [85]:
previous_agg.head()

In [37]:
combined_bb_prev = combined_bb.join(previous_agg, how="left", on="SK_ID_CURR")

In [38]:
combined_bb_prev.head()

,index,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_M,CODE_GENDER_XNA,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,FLAG_PHONE,FLAG_WORK_PHONE,HOUR_APPR_PROCESS_START,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGA

In [39]:
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
var

[('previous_application', 2019680147),
 ('combined_bb_prev', 1353412849),
 ('POS_CASH_balance', 1192493216),
 ('credit_card_balance', 918225044),
 ('installments_payments', 870745768),
 ('application_train', 570142137),
 ('previous_application_one_hot', 532798370),
 ('previous_agg', 484226677),
 ('combined_bb', 455650249),
 ('application_test', 90020943),
 ('sample_submission', 780008),
 ('bureau_balance_dummy_cat', 1928),
 ('atrcols', 1704),
 ('atcols', 1664),
 ('previous_application_cat', 1248),
 ('application_cat', 1072),
 ('application_test_cat', 1072),
 ('Imputer', 1056),
 ('KFold', 1056),
 ('KNN', 1056),
 ('LogisticRegression', 1056),
 ('MinMaxScaler', 1056),
 ('PCA', 1056),
 ('StandardScaler', 1056),
 ('StratifiedKFold', 1056),
 ('var', 736),
 ('columns', 440),
 ('In', 432),
 ('Out', 368),
 ('bureau_imp_dummy_cat', 264),
 ('color', 176),
 ('contextmanager', 136),
 ('cross_val_predict', 136),
 ('cross_val_score', 136),
 ('delete_columns', 136),
 ('feature_engineering_POS_CASH_bal

In [40]:
del previous_application
gc.collect()

90

In [ ]:
missing_value_summary(combined_bb_prev)

In [ ]:
POS_CASH_balance.head()

In [41]:
POS_CASH_balance_one_hot, POS_cat = one_hot_encoder(POS_CASH_balance)

In [ ]:
POS_cat

In [42]:
POS_agg= feature_engineering_POS_CASH_balance(POS_CASH_balance_one_hot, POS_cat)

In [ ]:
POS_agg.head()

In [43]:
combined_bb_prev_POS = combined_bb_prev.join(POS_agg, how='left', on='SK_ID_CURR')

In [44]:
del POS_agg, POS_CASH_balance_one_hot, combined_bb_prev, POS_CASH_balance
gc.collect()

6170

In [ ]:
installments_payments.head()

In [45]:
installments_agg= feature_engineering_installments(installments_payments)

In [ ]:
installments_agg.head()

In [46]:
combined_bb_prev_POS_ins = combined_bb_prev_POS.join(installments_agg,
                                                     how='left', on='SK_ID_CURR')

In [ ]:
combined_bb_prev_POS_ins.head()

In [47]:
del installments_agg, combined_bb_prev_POS, installments_payments
gc.collect()

21

In [ ]:
credit_card_balance.head()

In [ ]:
missing_value_summary(credit_card_balance)

In [48]:
credit_agg= feature_engineering_credit(credit_card_balance)

In [ ]:
credit_agg.head()

In [49]:
combined_bb_prev_POS_ins_cc= combined_bb_prev_POS_ins.join(credit_agg, 
                                                           how='left' , on='SK_ID_CURR' )

In [ ]:
missing_value_summary(combined_bb_prev_POS_ins_cc)

In [50]:
del combined_bb_prev_POS_ins, application_train, application_test
gc.collect()

37

In [52]:
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
var

[('combined_bb_prev_POS_ins_cc', 1789468969),
 ('credit_card_balance', 918225044),
 ('previous_application_one_hot', 532798370),
 ('previous_agg', 484226677),
 ('combined_bb', 455650249),
 ('credit_agg', 88645672),
 ('sample_submission', 780008),
 ('bureau_balance_dummy_cat', 1928),
 ('atrcols', 1704),
 ('atcols', 1664),
 ('previous_application_cat', 1248),
 ('application_cat', 1072),
 ('application_test_cat', 1072),
 ('Imputer', 1056),
 ('KFold', 1056),
 ('KNN', 1056),
 ('LogisticRegression', 1056),
 ('MinMaxScaler', 1056),
 ('PCA', 1056),
 ('StandardScaler', 1056),
 ('StratifiedKFold', 1056),
 ('var', 744),
 ('Out', 648),
 ('In', 528),
 ('columns', 440),
 ('bureau_imp_dummy_cat', 264),
 ('POS_cat', 192),
 ('color', 176),
 ('contextmanager', 136),
 ('cross_val_predict', 136),
 ('cross_val_score', 136),
 ('delete_columns', 136),
 ('feature_engineering_POS_CASH_balance', 136),
 ('feature_engineering_bureau', 136),
 ('feature_engineering_bureau_balance', 136),
 ('feature_engineering_cred

In [53]:
del credit_card_balance, credit_agg, previous_application_one_hot, previous_agg
gc.collect()

6560

In [55]:
combined_bb_prev_POS_ins_cc.to_csv("dataset.csv")

In [56]:
del combined_bb_prev_POS_ins_cc
gc.collect()

5171

In [58]:
del combined_bb
gc.collect()

7

In [60]:
del bureau_balance_dummy_cat
gc.collect()

898

In [54]:
train_X, test_X, train_y= preprocessing_imputer(combined_bb_prev_POS_ins_cc)

In [61]:
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
var

[('train_X', 1776183648),
 ('test_X', 281545456),
 ('train_y', 2460192),
 ('sample_submission', 780008),
 ('atrcols', 1704),
 ('atcols', 1664),
 ('previous_application_cat', 1248),
 ('Imputer', 1184),
 ('Out', 1184),
 ('application_cat', 1072),
 ('application_test_cat', 1072),
 ('KFold', 1056),
 ('KNN', 1056),
 ('LogisticRegression', 1056),
 ('MinMaxScaler', 1056),
 ('PCA', 1056),
 ('StandardScaler', 1056),
 ('StratifiedKFold', 1056),
 ('var', 720),
 ('In', 640),
 ('columns', 440),
 ('bureau_imp_dummy_cat', 264),
 ('POS_cat', 192),
 ('color', 176),
 ('contextmanager', 136),
 ('cross_val_predict', 136),
 ('cross_val_score', 136),
 ('delete_columns', 136),
 ('feature_engineering_POS_CASH_balance', 136),
 ('feature_engineering_bureau', 136),
 ('feature_engineering_bureau_balance', 136),
 ('feature_engineering_credit', 136),
 ('feature_engineering_installments', 136),
 ('feature_engineering_previous', 136),
 ('imputation', 136),
 ('impute_median', 136),
 ('init_notebook_mode', 136),
 ('ipl

In [64]:
minim=np.finfo(np.float64).min
maxim=np.finfo(np.float64).max
# train_X[0,0].dtype

In [65]:
(train_X >= maxim).all() and (train_X <= minim).all()

False

In [68]:
te=np.isinf(test_X)

In [70]:
test_X[te]=1e10
test_X[te]

array([1.e+10, 1.e+10, 1.e+10])

In [71]:
tr=np.isinf(train_X)

In [72]:
train_X[tr]=1e10
train_X[tr]

array([1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10,
       1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10, 1.e+10])

In [74]:
print(train_X.shape, test_X.shape)

(307511, 722) (48744, 722)


In [75]:
np.isinf(train_X).all()

False

In [76]:
del tr, te
gc.collect()

73

In [77]:
train_X, test_X= preprocessing_scaler(train_X, test_X)

In [80]:
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# Get a sorted list of the objects and their sizes
var=sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')], key=lambda x: x[1], reverse=True)
var

[('train_X', 1776183648),
 ('test_X', 281545456),
 ('train_y', 2460192),
 ('sample_submission', 780008),
 ('atrcols', 1704),
 ('atcols', 1664),
 ('LogisticRegression', 1464),
 ('previous_application_cat', 1248),
 ('Imputer', 1184),
 ('Out', 1184),
 ('application_cat', 1072),
 ('application_test_cat', 1072),
 ('KFold', 1056),
 ('KNN', 1056),
 ('MinMaxScaler', 1056),
 ('PCA', 1056),
 ('StratifiedKFold', 1056),
 ('StandardScaler', 888),
 ('In', 768),
 ('var', 712),
 ('columns', 440),
 ('bureau_imp_dummy_cat', 264),
 ('POS_cat', 192),
 ('color', 176),
 ('contextmanager', 136),
 ('cross_val_predict', 136),
 ('cross_val_score', 136),
 ('delete_columns', 136),
 ('feature_engineering_POS_CASH_balance', 136),
 ('feature_engineering_bureau', 136),
 ('feature_engineering_bureau_balance', 136),
 ('feature_engineering_credit', 136),
 ('feature_engineering_installments', 136),
 ('feature_engineering_previous', 136),
 ('imputation', 136),
 ('impute_median', 136),
 ('init_notebook_mode', 136),
 ('iplo

In [201]:
train_X.shape

(307511, 722)

In [ ]:
sum(sum(train_X <= minim))#or (train_X <= -1e10).all()

In [133]:
scores=[]  
a=time.clock()
for cv in range(1,6):
    print(cv)
    score=logistic_regression(train_X[:10000],train_y[:10000], cv=10)
    scores.append(score)
b=time.clock()
b-a

1
2
3
4
5


637.924137

In [153]:
mean_scores=[]
for arr in scores:
    mean_scores.append(arr.mean())
mean_scores

[]

In [184]:
 
a=time.clock()

rfmodel=random_forest(train_X[:10000],train_y[:10000], cv=5)

b=time.clock()
b-a

6.305234999999811

In [202]:
comb=pd.read_csv("dataset.csv")

In [190]:
comb0=comb.drop(['TARGET','SK_ID_CURR', 'index'],axis=1)

In [192]:
features= list(comb0.columns)
features

['Unnamed: 0',
 'AMT_ANNUITY',
 'AMT_CREDIT',
 'AMT_GOODS_PRICE',
 'AMT_INCOME_TOTAL',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'CNT_CHILDREN',
 'CNT_FAM_MEMBERS',
 'CODE_GENDER_M',
 'CODE_GENDER_XNA',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_REGISTRATION',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'FLAG_CONT_MOBILE',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_EMAI

In [199]:
len(rfmodel.feature_importances_)

722

In [193]:
del comb0, comb
gc.collect()

3056

In [200]:
len(features)

723

--------------------

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Ditribution of credit amount")
ax=sns.distplot(application_train["AMT_CREDIT"] )

In [ ]:
plt.figure(figsize=(20,12))
plt.title("Distribution for income")
ax=sns.distplot(application_train["AMT_INCOME_TOTAL"])

In [ ]:
plt.figure(figsize=(20,12))
plt.title("Distribution for income")
plt.boxplot(application_train["AMT_INCOME_TOTAL"])

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Distribution - AMT_GOODS_PRICE")
ax=sns.distplot(application_train["AMT_GOODS_PRICE"].dropna())

In [ ]:
temp = application_train["NAME_TYPE_SUITE"].value_counts()
#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp / temp.sum())*100,
)
data = [trace]
layout = go.Layout(
    title = "Who accompanied client when applying for the  application in % ",
    xaxis=dict(
        title='Name of type of the Suite',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of Name of type of the Suite in %',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='schoolStateNames')

In [ ]:
temp = application_train["TARGET"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
df.iplot(kind='pie',labels='labels',values='values', title='Loan Repayed or not')

In [ ]:
temp = application_train["NAME_CONTRACT_TYPE"].value_counts()
fig = {
  "data": [
    {
      "values": temp.values,
      "labels": temp.index,
      "domain": {"x": [0, .48]},
      #"name": "Types of Loans",
      #"hoverinfo":"label+percent+name",
      "hole": .7,
      "type": "pie"
    },
    
    ],
  "layout": {
        "title":"Types of loan",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Loan Types",
                "x": 0.17,
                "y": 0.5
            }
            
        ]
    }
}
iplot(fig, filename='donut')

In [ ]:
temp1 = application_train["FLAG_OWN_CAR"].value_counts()
temp2 = application_train["FLAG_OWN_REALTY"].value_counts()

fig = {
  "data": [
    {
      "values": temp1.values,
      "labels": temp1.index,
      "domain": {"x": [0, .48]},
      "name": "Own Car",
      "hoverinfo":"label+percent+name",
      "hole": .6,
      "type": "pie"
    },
    {
      "values": temp2.values,
      "labels": temp2.index,
      "text":"Own Realty",
      "textposition":"inside",
      "domain": {"x": [.52, 1]},
      "name": "Own Reality",
      "hoverinfo":"label+percent+name",
      "hole": .6,
      "type": "pie"
    }],
  "layout": {
        "title":"Purpose of loan",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Own Car",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Own Realty",
                "x": 0.8,
                "y": 0.5
            }
        ]
    }
}
iplot(fig, filename='donut')

### 4. Exploratory Data Analysis

In [ ]:
from sklearn import preprocessing
cat_features = [f for f in application_train.columns if application_train[f].dtype=='object']

for col in cat_features:
    lb=preprocessing.LabelEncoder()
    lb.fit(list(application_train[col].values.astype('str')) + list(application_test[col].values.astype('str')) )
    application_train[col]=lb.transform(list(application_train[col].values.astype('str')) )
    application_test[col]=lb.transform(list(application_test[col].values.astype('str')) )
    


In [ ]:
appy=application_train.fillna(-999)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier(n_estimators=50, max_depth=8, min_samples_leaf=4, max_features=0.5, random_state=2018)
rf.fit(appy.drop(['SK_ID_CURR','TARGET'],axis=1), appy.TARGET)
features=appy.drop(['SK_ID_CURR','TARGET'], axis=1).columns.values

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(rf.feature_importances_, features), 
                                                            reverse = False)))
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='Random Forest Feature importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
#         domain=[0, 0.85],
    ),
    margin=dict(
    l=300,
),
)

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

In [ ]:
x

In [ ]:
y